In [1]:
#combined_df["purchaseToTripRatio"] = combined_df["daysSinceLastPurchase"] / combined_df["avgDaysBetweenPurchases"]


# encoded_df["due_score"] = (
#     1.5 * encoded_df["daysSinceLastPurchase_norm"]
#   + 1.0 * encoded_df["freq_30_norm"]
#   + 0.5 * encoded_df["freq_90_norm"]
# )

#encoded_df["due_score"] = 1 / (1 + np.exp(-encoded_df["due_score"]))

# df["bulkAdjustedUrgencyRatio"] = df.apply(
#     lambda row: BulkPurchaseFeatures.bulk_adjusted_urgency_ratio(
#         row["daysSinceLastPurchase"],
#         row["avgDaysBetweenPurchases"],
#         row["bulkFlag"],
#         row["didBuy"]          # this is already 1 or 0 per trip
#     ),
#     axis=1
# )


In [2]:
import pandas as pd
from pathlib import Path
from typing import List, Dict, Any, Optional, Tuple
import os
from datetime import datetime
import pdb;
from grocery_ml import GroceryML
from hidden_layer_param_builder import HiddenLayerParamSetBuilder

pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)
pd.set_option("display.float_format", lambda x: f"{x:.6f}")
pd.set_option("display.max_columns", None)
pd.set_option("display.width", 2000)

print(os.getcwd())
# print("GPUs Available:", tf.config.list_physical_devices('GPU'))
#tf.debugging.set_log_device_placement(True)

def run_all_experiments(df, model_param_sets, output_dir):
    total = len(model_param_sets)
    print(f"run_all_experiments() when: {datetime.now()}  output_dir: {output_dir}");
    for index, params in enumerate(model_param_sets, 1):
        print(f"Running Exp {index}/{total}...")
        groceryML.run_experiment(df,  params["buildParams"], params["trainParams"], output_dir)



groceryML = GroceryML();
groceryML.build_combined_df(use_neg_samples=True)
if groceryML.combined_df is None:
    raise();

C:\Users\steve\source\repos\grocery-ml
build_combined_df(use_neg_samples = True)
creating target col: didBuy_target
use_neg_samples is true
insert_negative_samples()
build_purchase_item_freq_cols()
build_habit_frequency_for_training()
compute_habit_frequency_map()
build_trip_level_features()
validate_no_empty_columns()
build_combined_df() done


In [3]:
# quick test shot
layers = [
    { "units": 1, "activation": "relu" }
]
modelParamsList = []
modelParamsList.append({
    "trainParams": { "epochs": 1 },
    "buildParams": {
    "embedding_dim": 1,
    "layers": layers,
    "activation": "relu",
    "output_activation": "sigmoid",
    "optimizer": "adam",
    "learning_rate": 0.01,
    "loss": "binary_crossentropy",
    "metrics": "BinaryCrossentropy"
    }
})
ts = datetime.now().strftime("%Y_%m_%d_%H_%M_%S")
run_all_experiments(groceryML.combined_df, modelParamsList, f"exp/keras/keras-2/test-shot-no-trust-{ts}")

run_all_experiments() when: 2025-12-31 19:34:41.026924  output_dir: exp/keras/keras-2/test-shot-no-trust-2025_12_31_19_34_41
Running Exp 1/1...
Creating dir: exp/keras/keras-2/test-shot-no-trust-2025_12_31_19_34_41\e1__l1__ep1__oa_sigmoid__719723
run_experiment()  baseDir: exp/keras/keras-2/test-shot-no-trust-2025_12_31_19_34_41\e1__l1__ep1__oa_sigmoid__719723  
run_experiment()  when: 2025-12-31 19:34:41.031213 params: {'epochs': 1}  
normalize_features()
train_model()
 build_prediction_input()   Prediction date: 2025-12-31
normalize_features()
build_prediction_input() is done
Running Model.Predict()
23/23 [==============================] - 0s 713us/step
Exporting extra_dataframes:
Writing XLSX: exp/keras/keras-2/test-shot-no-trust-2025_12_31_19_34_41\e1__l1__ep1__oa_sigmoid__719723\latest_rows_df.xlsx
   XLSX Done: exp/keras/keras-2/test-shot-no-trust-2025_12_31_19_34_41\e1__l1__ep1__oa_sigmoid__719723\latest_rows_df.xlsx
Writing XLSX: exp/keras/keras-2/test-shot-no-trust-2025_12_31_

INFO:tensorflow:Assets written to: exp/keras/keras-2/test-shot-no-trust-2025_12_31_19_34_41\e1__l1__ep1__oa_sigmoid__719723\model\assets


INFO:tensorflow:Assets written to: exp/keras/keras-2/test-shot-no-trust-2025_12_31_19_34_41\e1__l1__ep1__oa_sigmoid__719723\model\assets


[save_model] saving model weights (separate file)
[save_model] all artifacts saved successfully → exp/keras/keras-2/test-shot-no-trust-2025_12_31_19_34_41\e1__l1__ep1__oa_sigmoid__719723\model
Saved experiment → exp/keras/keras-2/test-shot-no-trust-2025_12_31_19_34_41\e1__l1__ep1__oa_sigmoid__719723


In [ ]:
###########################################################

modelParamsList = []
layers = [
    { "units": 64, "activation": "linear" },
    { "units": 64, "activation": "relu" },
    { "units": 64, "activation": "relu" },
    { "units": 64, "activation": "relu" },
    { "units": 16, "activation": "relu" },
    { "units": 16, "activation": "relu" },
    { "units": 16, "activation": "relu" },
    { "units": 16, "activation": "relu" },
]
output_activations = ["sigmoid", "linear", "tanh", "relu"]
embDims = [12,24,36,48,64,128,256,512,1024]
metrics = ["Accuracy", "MAE", "MSE", "MAPE", "MSLE", "Precision", "Recall", "AUC",  "BinaryCrossentropy", "RootMeanSquaredError"]

for act in output_activations:
    for dim in embDims:
        modelParamsList.append({
            "trainParams": { "epochs": 20 },
            "buildParams": {
                "embedding_dim": dim,
                "layers": layers,
                "output_activation": act,
                "optimizer": "adam",
                "learning_rate": 0.0001,
                "loss": "binary_crossentropy" if act == "sigmoid" else "mse",
                "metrics": metrics
            }
        })

ts = datetime.now().strftime("%Y_%m_%d_%H_%M_%S")
run_all_experiments(groceryML.combined_df, modelParamsList, f"exp/keras/keras-2/{ts}")

run_all_experiments() when: 2025-12-31 19:37:31.217601  output_dir: exp/keras/keras-2/2025_12_31_19_37_31
Running Exp 1/36...
Creating dir: exp/keras/keras-2/2025_12_31_19_37_31\e12__l64-64-64-64-16-16-16-16__ep20__oa_sigmoid__504065
run_experiment()  baseDir: exp/keras/keras-2/2025_12_31_19_37_31\e12__l64-64-64-64-16-16-16-16__ep20__oa_sigmoid__504065  
run_experiment()  when: 2025-12-31 19:37:31.218985 params: {'epochs': 20}  
normalize_features()
train_model()
 build_prediction_input()   Prediction date: 2025-12-31
normalize_features()
build_prediction_input() is done
Running Model.Predict()
23/23 [==============================] - 0s 636us/step
Exporting extra_dataframes:
Writing XLSX: exp/keras/keras-2/2025_12_31_19_37_31\e12__l64-64-64-64-16-16-16-16__ep20__oa_sigmoid__504065\latest_rows_df.xlsx
   XLSX Done: exp/keras/keras-2/2025_12_31_19_37_31\e12__l64-64-64-64-16-16-16-16__ep20__oa_sigmoid__504065\latest_rows_df.xlsx
Writing XLSX: exp/keras/keras-2/2025_12_31_19_37_31\e12__l6

INFO:tensorflow:Assets written to: exp/keras/keras-2/2025_12_31_19_37_31\e12__l64-64-64-64-16-16-16-16__ep20__oa_sigmoid__504065\model\assets


INFO:tensorflow:Assets written to: exp/keras/keras-2/2025_12_31_19_37_31\e12__l64-64-64-64-16-16-16-16__ep20__oa_sigmoid__504065\model\assets


[save_model] saving model weights (separate file)
[save_model] all artifacts saved successfully → exp/keras/keras-2/2025_12_31_19_37_31\e12__l64-64-64-64-16-16-16-16__ep20__oa_sigmoid__504065\model
Saved experiment → exp/keras/keras-2/2025_12_31_19_37_31\e12__l64-64-64-64-16-16-16-16__ep20__oa_sigmoid__504065
Running Exp 2/36...
Creating dir: exp/keras/keras-2/2025_12_31_19_37_31\e24__l64-64-64-64-16-16-16-16__ep20__oa_sigmoid__188250
run_experiment()  baseDir: exp/keras/keras-2/2025_12_31_19_37_31\e24__l64-64-64-64-16-16-16-16__ep20__oa_sigmoid__188250  
run_experiment()  when: 2025-12-31 19:41:25.816406 params: {'epochs': 20}  
normalize_features()
train_model()
 build_prediction_input()   Prediction date: 2025-12-31
normalize_features()
build_prediction_input() is done
Running Model.Predict()
23/23 [==============================] - 0s 728us/step
Exporting extra_dataframes:
Writing XLSX: exp/keras/keras-2/2025_12_31_19_37_31\e24__l64-64-64-64-16-16-16-16__ep20__oa_sigmoid__188250\la

INFO:tensorflow:Assets written to: exp/keras/keras-2/2025_12_31_19_37_31\e24__l64-64-64-64-16-16-16-16__ep20__oa_sigmoid__188250\model\assets


...

[save_model] saving model weights (separate file)
[save_model] all artifacts saved successfully → exp/keras/keras-2/2025_12_31_19_37_31\e24__l64-64-64-64-16-16-16-16__ep20__oa_tanh__273970\model
Saved experiment → exp/keras/keras-2/2025_12_31_19_37_31\e24__l64-64-64-64-16-16-16-16__ep20__oa_tanh__273970
Running Exp 21/36...
Creating dir: exp/keras/keras-2/2025_12_31_19_37_31\e36__l64-64-64-64-16-16-16-16__ep20__oa_tanh__205600
run_experiment()  baseDir: exp/keras/keras-2/2025_12_31_19_37_31\e36__l64-64-64-64-16-16-16-16__ep20__oa_tanh__205600  
run_experiment()  when: 2025-12-31 21:38:14.891649 params: {'epochs': 20}  
normalize_features()
train_model()


ERROR:asyncio:Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "C:\ProgramData\miniconda3\envs\grocery-ml\lib\asyncio\events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
  File "C:\ProgramData\miniconda3\envs\grocery-ml\lib\asyncio\selector_events.py", line 115, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host
ERROR:asyncio:Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "C:\ProgramData\miniconda3\envs\grocery-ml\lib\asyncio\events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
  File "C:\ProgramData\miniconda3\envs\grocery-ml\lib\asyncio\selector_events.py", line 115, in _read_from_self
    data = self._ssoc

 build_prediction_input()   Prediction date: 2025-12-31
normalize_features()
build_prediction_input() is done
Running Model.Predict()
 1/23 [>.............................] - ETA: 1s

ERROR:asyncio:Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "C:\ProgramData\miniconda3\envs\grocery-ml\lib\asyncio\events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
  File "C:\ProgramData\miniconda3\envs\grocery-ml\lib\asyncio\selector_events.py", line 115, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host
ERROR:asyncio:Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "C:\ProgramData\miniconda3\envs\grocery-ml\lib\asyncio\events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
  File "C:\ProgramData\miniconda3\envs\grocery-ml\lib\asyncio\selector_events.py", line 115, in _read_from_self
    data = self._ssoc

 2/23 [=>............................] - ETA: 2:22